<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ALIGNN_FFvsM3GNET_on_MLEARN_ALIGNNFF_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install alignn m3gnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.8/973.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import site
site.main()

from pymatgen.core import Structure, Lattice
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import numpy as np
import json
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import functools
from matplotlib import cm
import os
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['axes.axisbelow'] = True
mpl.rcParams['xtick.labelsize'] = 24
mpl.rcParams['ytick.labelsize'] = 24

In [3]:
# get data from mlearn
#!rm *.json
!for m in Ni Cu Mo Ge Si Li; do wget https://github.com/materialsvirtuallab/mlearn/raw/master/data/${m}/test.json; mv test.json ${m}_test.json; done;
!ls

--2023-04-30 20:32:13--  https://github.com/materialsvirtuallab/mlearn/raw/master/data/Ni/test.json
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/materialsvirtuallab/mlearn/master/data/Ni/test.json [following]
--2023-04-30 20:32:13--  https://raw.githubusercontent.com/materialsvirtuallab/mlearn/master/data/Ni/test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611126 (597K) [text/plain]
Saving to: ‘test.json’

test.json           100%[===================>] 596.80K  --.-KB/s    in 0.02s   

2023-04-30 20:32:13 (31.0 MB/s) - ‘test.json’ saved [611126/611126]

--2023-04-30 20:32:13--  https

In [5]:
from ase import Atoms
import numpy as np
import torch
torch.cuda.is_available = lambda : False
import pymatgen.io.ase as aio
from alignn.ff.ff import AlignnAtomwiseCalculator,default_path
model_path = default_path()
calc = AlignnAtomwiseCalculator(path=model_path,stress_wt=0.3)

model_path /usr/local/lib/python3.10/dist-packages/alignn/ff


In [ ]:

material_dict = {'Ni': 'Nickel', 'Cu': 'Copper', 'Mo': 'Molybdenum', 'Ge': 'Germanium', 'Si': 'Silicon', 'Li': 'Lithium'}

for m in ['Ni', 'Cu', 'Mo', 'Ge', 'Si', 'Li']:
  f = open(f'{m}_test.json')
  test = json.load(f)

  e_true = []
  e_pred = []
  f_true = []
  f_pred = []
  n_atoms_all = []

  for ex in tqdm(test):
    s = Structure.from_dict(ex['structure'])
    atoms = aio.AseAtomsAdaptor.get_atoms(s)

    # TODO: understand why this setting tags is required
    atoms.set_tags(np.ones(len(atoms)))
    atoms.calc = calc
    e = atoms.get_potential_energy()
    f = atoms.get_forces()

    e_true.append(ex['outputs']['energy'])
    e_pred.append(e)

    t = np.array(ex['outputs']['forces'])

    f_true.append(t)
    f_pred.append(f)

  f_true_ = np.concatenate(f_true).flatten()
  f_pred_ = np.concatenate(f_pred).flatten()
  print(f'Material: {m}, F-RMSE: {np.sqrt(np.mean(np.power(f_true_ - f_pred_, 2)))}, F-MAE: {np.mean(np.abs(f_true_ - f_pred_))}')

  plt.figure(figsize=(15, 8))
  matplotlib.pyplot.grid(True, which="both")
  plt.scatter(f_true_, f_pred_, s=10)
  plt.plot(f_true_, f_true_, 'k--')
  _ = plt.xlabel('True Force Component, [eV/A]', fontsize=24)
  _ = plt.ylabel('Predicted Force Component, [eV/A]', fontsize=24)
  plt.title(f'{material_dict[m]}, ALIGNN-FF, Zero-shot\nF-MAE: {1000 * np.mean(np.abs(f_true_ - f_pred_)):.3f} meV/A, F-RMSE: {1000 * np.sqrt(np.mean(np.power(f_true_ - f_pred_, 2))):.3f} meV/A', fontsize=24)
  plt.show()

 10%|▉         | 3/31 [01:19<12:23, 26.54s/it]

In [ ]:
from m3gnet.models import M3GNet, M3GNetCalculator, Potential
potential = Potential(M3GNet.load())
calculator = M3GNetCalculator(potential=potential, stress_weight=0.01)
material_dict = {'Ni': 'Nickel', 'Cu': 'Copper', 'Mo': 'Molybdenum', 'Ge': 'Germanium', 'Si': 'Silicon', 'Li': 'Lithium'}

for m in ['Ni', 'Cu', 'Mo', 'Ge', 'Si', 'Li']:
  f = open(f'{m}_test.json')
  test = json.load(f)

  e_true = []
  e_pred = []
  f_true = []
  f_pred = []
  n_atoms_all = []

  for ex in tqdm(test):
    s = Structure.from_dict(ex['structure'])
    atoms = aio.AseAtomsAdaptor.get_atoms(s)

    # TODO: understand why this setting tags is required
    atoms.set_tags(np.ones(len(atoms)))
    atoms.calc = calculator
    e = atoms.get_potential_energy()
    f = atoms.get_forces()

    e_true.append(ex['outputs']['energy'])
    e_pred.append(e)

    t = np.array(ex['outputs']['forces'])

    f_true.append(t)
    f_pred.append(f)

  f_true_ = np.concatenate(f_true).flatten()
  f_pred_ = np.concatenate(f_pred).flatten()
  print(f'Material: {m}, F-RMSE: {np.sqrt(np.mean(np.power(f_true_ - f_pred_, 2)))}, F-MAE: {np.mean(np.abs(f_true_ - f_pred_))}')

  plt.figure(figsize=(15, 8))
  matplotlib.pyplot.grid(True, which="both")
  plt.scatter(f_true_, f_pred_, s=10)
  plt.plot(f_true_, f_true_, 'k--')
  _ = plt.xlabel('True Force Component, [eV/A]', fontsize=24)
  _ = plt.ylabel('Predicted Force Component, [eV/A]', fontsize=24)
  plt.title(f'{material_dict[m]}, MEGNET, Zero-shot\nF-MAE: {1000 * np.mean(np.abs(f_true_ - f_pred_)):.3f} meV/A, F-RMSE: {1000 * np.sqrt(np.mean(np.power(f_true_ - f_pred_, 2))):.3f} meV/A', fontsize=24)
  plt.show()

In [ ]:
from jarvis.db.figshare import data
ff_db=data('alignn_ff_db')
print(len(ff_db))

In [ ]:
from alignn.ff.ff import default_path,ForceField
from alignn.graphs import Graph
import torch
import os
from jarvis.db.jsonutils import loadjson
from alignn.models.alignn_atomwise import ALIGNNAtomWise,ALIGNNAtomWiseConfig
torch.cuda.is_available = lambda : False
from jarvis.core.atoms import Atoms
from tqdm import tqdm
import numpy as np
from alignn.ff.ff import default_path,ForceField
from jarvis.core.atoms import Atoms
from alignn.ff.ff import AlignnAtomwiseCalculator,default_path
from ase.stress import full_3x3_to_voigt_6_stress
import torch


model_path = default_path()
calc = AlignnAtomwiseCalculator(path=model_path,force_mult_natoms=True)


def get_alignn_forces(atoms=[]):
    ase_atoms=atoms.ase_converter()
    ase_atoms.calc = calc
    e = ase_atoms.get_potential_energy()
    f = ase_atoms.get_forces()
    s = ase_atoms.get_stress()
    return e,f,s

torch.cuda.is_available = lambda : False
mg=[]

al_e=[]
al_f=[]
al_s=[]

t_e=[]
t_f=[]
t_s=[]
#Taking last 25 samples for quick testing
for ii,i in tqdm(enumerate(ff_db[-25:])):
    #if ii<10:
        atoms = Atoms.from_dict(i['atoms'])
        #mg_f = get_m3gnet_forces(atoms)
        #mg.append(np.array(mg_f).flatten())
        e,f,s = get_alignn_forces(atoms)
        al_e.append(e)
        t_e.append(i['total_energy']*atoms.num_atoms)
        
        al_f.append(np.array(f).flatten())
        t_f.append(np.array(i['forces']).flatten())
        ts=full_3x3_to_voigt_6_stress(i['stresses'])
        al_s.append(np.array(s).flatten())
        t_s.append(np.array(ts).flatten())
        


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(np.concatenate(t_f).flatten(),np.concatenate(al_f).flatten(),'.')
plt.xlabel('DFT Forces(eV/A)')
plt.ylabel('ALIGNN-FF Forces(eV/A)')

In [ ]:
plt.plot(t_e,al_e,'.')
plt.xlabel('DFT Energy(eV)')
plt.ylabel('ALIGNN-FF Energy(eV)')

In [ ]:
plt.plot(np.concatenate(t_s),-1600*np.concatenate(al_s),'.')
plt.xlabel('DFT Stress(eV/A2)')
plt.ylabel('ALIGNN-FF Stress(eV/A2)')